In [22]:
import os
from dotenv import load_dotenv
import openai
# Load environment variables from .env file
from autogen import  AssistantAgent
from autogen.agentchat.contrib.llamaindex_conversable_agent import LLamaIndexConversableAgent
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex, GPTVectorStoreIndex
from llama_index.embeddings.openai import OpenAIEmbedding
from dotenv import load_dotenv
from llama_index.agent.openai import OpenAIAgent
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.agent import AgentRunner
from llama_index.llms import openai
from llama_index.core import VectorStoreIndex, get_response_synthesizer
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.retrievers.bm25 import BM25Retriever
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.retrievers import QueryFusionRetriever
from llama_index.core.tools import QueryEngineTool, ToolMetadata
from autogen import ConversableAgent, UserProxyAgent
from autogen.agentchat.contrib.capabilities import teachability
from autogen import Cache
from autogen.agentchat.contrib.capabilities.text_compressors import LLMLingua
from autogen.agentchat.contrib.capabilities.transforms import TextMessageCompressor
from autogen.agentchat.contrib.capabilities import transform_messages
import autogen
import nest_asyncio
from llama_parse import LlamaParse
import Stemmer
from autogen import ConversableAgent, UserProxyAgent
from llama_index.core.agent import (
    StructuredPlannerAgent,
    FunctionCallingAgentWorker,
    ReActAgentWorker,
)
nest_asyncio.apply()
load_dotenv()

from huggingface_hub import login

hugging_face_api = os.getenv('HUGGINGFACE_API_KEY')
login(hugging_face_api)

openai_api_key = os.getenv("OPENAI_API_KEY")
LLAMA_CLOUD_KEY = os.getenv("LLAMA_CLOUD_KEY")


In [15]:

# Load and process documents (ensure you have a `documents/` folder)


parser = LlamaParse(
    api_key=LLAMA_CLOUD_KEY,  # can also be set in your env as LLAMA_CLOUD_API_KEY
    result_type="markdown",  # "markdown" and "text" are available
    verbose=True,
)

file_extractor = {".pdf": parser}
documents = SimpleDirectoryReader(input_dir=r"Data_repo", file_extractor=file_extractor).load_data()

# Create a vector index


Started parsing the file under job_id 45ef43b8-4816-4009-aa04-d08fdd012ace
.Started parsing the file under job_id a7c823e8-08f6-4de2-9395-e56648002ba2
.Started parsing the file under job_id bb67ab19-ee67-47c0-a7aa-8d008d40bfb1
.Started parsing the file under job_id 9ac817e6-9b77-4314-bfb5-1c917cf082c9
Started parsing the file under job_id 2a2c553b-19f1-4ccf-9807-b7fc4ac56df8
.

Creating normal agent from vector index retriever

In [17]:
llm =  openai.OpenAI()
index = GPTVectorStoreIndex.from_documents(documents)
retriever = VectorIndexRetriever(index=index, similarity_top_k=3)

response_synthesizer = get_response_synthesizer(
    response_mode="tree_summarize",
)
query_engine = RetrieverQueryEngine(
    retriever=retriever,
    response_synthesizer=response_synthesizer,
)
retrieval_tool = QueryEngineTool(
    query_engine=query_engine,
    metadata=ToolMetadata(name="document_retriever", description="Retrieves relevant document sections"),
)


agent = AgentRunner.from_llm([retrieval_tool], llm=llm, verbose=True)

Creating Structured Planner Agent tool

In [18]:
#First create two different types of retrievers

from llama_index.retrievers.bm25 import BM25Retriever

# 1st is vector store
vector_retriever = VectorIndexRetriever(index=index, similarity_top_k=3)

# 2nd is bm25 retriever
bm25_retriever = BM25Retriever.from_defaults(
    docstore=index.docstore, similarity_top_k=2,
    stemmer=Stemmer.Stemmer("english"),
    language="english",
)


# Next, we can create our fusion retriever, which well return the top-2 most similar nodes from the 4 returned nodes from the retrievers:
query_fusion_retriever = QueryFusionRetriever(
    [vector_retriever, bm25_retriever],
    similarity_top_k=2,
    num_queries=4,  # set this to 1 to disable query generation
    mode="reciprocal_rerank",
    use_async=True,
    verbose=True,
    # query_gen_prompt="...",  # we could override the query generation prompt here
)

# Creating query engine from the retriever
advanced_retriever_query_engine = RetrieverQueryEngine.from_args(query_fusion_retriever)


# creating tool out of the query engine
advanced_retrieval_tool = QueryEngineTool(
    query_engine=advanced_retriever_query_engine,
    metadata=ToolMetadata(name="advanced_retriever", description="Retrieves relevant document sections for advanced queries"),
)



Segment to create agent worker and agents for advanced search

In [19]:

# create the function calling worker for reasoning
worker = FunctionCallingAgentWorker.from_tools(
    [advanced_retrieval_tool, retrieval_tool], verbose=True
)

# wrap the worker in the top-level planner
structured_agent = StructuredPlannerAgent(
    worker, tools= [advanced_retrieval_tool, retrieval_tool], verbose=True
)

Autogen agent with normal llamaindex agent

In [23]:
# Define LlamaIndexConversableAgent with RAG capabilities


llama_index_agent = LLamaIndexConversableAgent(
    name="LlamaIndexAgent",
    description='llamaindex agent to query information from the vector store',
    llama_index_agent=agent, # Uses LlamaIndex for retrieval
    llm_config={"model": "gpt-4", "api_key": openai_api_key}
)

# Define an AI assistant for responses
# assistant = AssistantAgent(
#     name="AI_Assistant",
#     llm_config={"model": "gpt-4", "api_key": openai_api_key}
# )

human_proxy = ConversableAgent(
    "human_proxy",
    llm_config=False,  # no LLM used for human proxy
    human_input_mode="ALWAYS",  # always ask for human input,
    is_termination_msg=lambda msg: "good bye" in msg["content"].lower() or None
)

Autogen agent with advanced llamaindex agent

In [24]:
llama_index_agent = LLamaIndexConversableAgent(
    name="LlamaIndexAgent",
    description='llamaindex agent to query information from the vector store',
    llama_index_agent=structured_agent, # Uses LlamaIndex for retrieval
    llm_config={"model": "gpt-4", "api_key": openai_api_key}
)

In [25]:
llm_lingua = LLMLingua()

In [ ]:
with Cache.disk(cache_path_root="/tmp/autogen_cache") as cache:
    
    
    text_compressor = TextMessageCompressor(
        text_compressor=llm_lingua,
        compression_params={"target_token": 13000},
        cache=cache,
    )
    context_handling = transform_messages.TransformMessages(transforms=[text_compressor])
    context_handling.add_to_agent(llama_index_agent)

    result = human_proxy.initiate_chat(llama_index_agent, message='what is the pdf about?', cache=cache)
    result.summary()

human_proxy (to LlamaIndexAgent):

what is the pdf about?

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
=== Initial plan ===
Retrieve Relevant Document Sections:
Provide the PDF document -> Relevant sections of the document
deps: []


Advanced Query:
Relevant sections of the document -> Detailed information about the content
deps: ['Retrieve Relevant Document Sections']


> Running step c2936f65-4558-4dbf-b3d2-9968f330ccb6. Step input: Provide the PDF document
Added user message to memory: Provide the PDF document
=== LLM Response ===
Please provide the PDF document that you would like me to analyze.
=== Refined plan ===
Advanced Query:
Relevant sections of the document -> Detailed information about the content
deps: ['Retrieve Relevant Document Sections']


> Running step 2e2318b6-a2f2-4d5d-b038-779ccfba52d6. Step input: Relevant sections of the document
Added user message to memory: Relevant sections of the document
==